In [1]:
#default_exp utils.math.compute_pillars

#  Compute pillars


> First step is computing the all necessary information regarding the pillars on the given point cloud. As source we used the original implementation in second (https://github.com/traveller59/second.pytorch/tree/master/second) as well as another implementation in tensorflow (https://github.com/fferroni/PointPillars). The main idea is to devide the point cloud into descrete sections, namly pillars, and calculate different attributes for each point in regards to the pillar it is in. These attributes include the distance to the pillar mean location as well as the distance to the pillar center.


## 00 - Prerequesits

In [2]:
import sys
#sys.path.append("/home/qhs67/git/bachelorthesis_sven_thaele/code/")
sys.path.insert(0, "../pointpillars")
from utils.time import time_method



#### 00.1 - Imports


In [3]:

#export
import logging
import torch
import numpy as np
from numba import njit

from utils.io import read_config

logger = logging.getLogger(__name__)



#### 00.2 - CUDA



Set the best cuda device to improve calculations


In [4]:

dt = torch.float32
dev = "cuda:0" if torch.cuda.is_available() else "cpu"
torch.cuda.set_device(dev)
#dev = "cpu"

dev

'cuda:0'


## 01 - Data structure



> Firstly, we want to familiarize ourselves with the structure of the binary files we receive from the
> pseudo-lidar convertion.


In [5]:
file_location_psl = "/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/pseudo_lidar/002000.bin"
file_location_lid = "/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/velodyne/training/000100.bin"

In [6]:
pc_vel = np.fromfile(file_location_lid, dtype=np.float32, count=-1)
pc_vel = pc_vel.reshape([-1,4])
pc_vel = torch.as_tensor(pc_vel, device=dev)
pc_vel, pc_vel.shape

(tensor([[43.8890,  0.0840,  1.6930,  0.0000],
         [43.2010,  0.2180,  1.6700,  0.0000],
         [42.3970,  0.3470,  1.6430,  0.0000],
         ...,
         [ 3.6960, -1.3980, -1.7260,  0.3100],
         [ 3.7170, -1.3990, -1.7360,  0.3500],
         [ 3.7240, -1.3890, -1.7370,  0.0000]], device='cuda:0'),
 torch.Size([123183, 4]))

In [7]:
pc_np = np.fromfile(file_location_psl, dtype=np.float32, count=-1)
pc = torch.as_tensor(pc_np, device=dev)
pc = pc.reshape([-1,4])
pc, pc.shape

(tensor([[ 4.9012, -0.1392,  0.9999,  1.0000],
         [ 4.9036, -0.2038,  0.9997,  1.0000],
         [ 4.8751, -0.2087,  0.9929,  1.0000],
         ...,
         [ 5.4258, -4.4040, -1.5072,  1.0000],
         [ 5.4245, -4.4102, -1.5069,  1.0000],
         [ 5.4078, -4.4028, -1.5023,  1.0000]], device='cuda:0'),
 torch.Size([279454, 4]))

In [8]:
x = pc[:,0]
y = pc[:,1]
z = pc[:,2]
r = pc[:,3]
x, y, z, r

(tensor([4.9012, 4.9036, 4.8751,  ..., 5.4258, 5.4245, 5.4078], device='cuda:0'),
 tensor([-0.1392, -0.2038, -0.2087,  ..., -4.4040, -4.4102, -4.4028],
        device='cuda:0'),
 tensor([ 0.9999,  0.9997,  0.9929,  ..., -1.5072, -1.5069, -1.5023],
        device='cuda:0'),
 tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0'))

## 02 - Methods

In [9]:
#export
def remove_invalid_points(pcloud: torch.tensor,
                          pillars_cfg: dict,
                          dt=torch.float32,
                          dev: str = "cuda:0"):
    """Removes invalid points exceeding the calculation bounds from the point clouds

    :param pcloud: Tensor containing the points from the point cloud
    :param pillars_cfg:

    :returns: pcloud without the points exceeding the calculation bounds
    """
    logger.info("Removing invalid points from point cloud...")

    min = torch.cuda.FloatTensor([pillars_cfg.getfloat("x_min"), pillars_cfg.getfloat("y_min"), pillars_cfg.getfloat("z_min")])
    max = torch.cuda.FloatTensor([pillars_cfg.getfloat("x_max"), pillars_cfg.getfloat("y_max"), pillars_cfg.getfloat("z_max")])

    xyz_points = pcloud[:,:3]
    mask = torch.logical_and(torch.le(min, xyz_points), torch.le(xyz_points, max))
    mask = torch.all(mask, dim=1)

    logger.debug(f"Removing complete.\n"
                 f"pcloud: {pcloud}{pcloud.shape}")
    return pcloud[mask]

In [10]:
arr = torch.rand([210000, 4], dtype=dt, device=dev, requires_grad=False) * 200 - 100
pillars_cfg = read_config()['pillars']

out = remove_invalid_points(arr, pillars_cfg)

out, out.shape

(tensor([[ 25.3190, -34.4627,   1.7883,  20.6380],
         [ 64.3730, -18.5386,  -0.7764,  93.3091],
         [ 20.4534, -36.8629,  -0.2945,  78.7316],
         ...,
         [ 27.9592, -39.8829,   1.4494,  36.2360],
         [  2.9664,   0.9887,  -0.1117, -68.9511],
         [ 68.0807, -29.9628,   1.0935,  20.1869]], device='cuda:0'),
 torch.Size([663, 4]))

In [11]:
arr = torch.rand([210000, 4], dtype=dt, device=dev) * 200 - 100
#pillars_cfg = read_config()['pillars']
time_method(remove_invalid_points, runs=1000, kwargs={"pcloud": arr, "pillars_cfg": pillars_cfg})

tensor(1.1098)

#### 02.1 - Get points in pillars method

> The methods to group the point cloud into pillars. It returns the points sorted into their specific pillar as well as
> setting the correct tensor dimensions via zero padding and random sampling, depending on the number of points per pillar.
> The configuration can be set in the config.ini file.

The method goes through the point cloud and assigns each point to its pillar. To improve performance for real time
application, we use numba. It translates the python code into c code just in time.

In [12]:
#export
@njit(parallel=False)
def _get_points_in_pillars(pcloud: np.ndarray,
                           pillars : np.ndarray,
                           pill_ind: np.ndarray,
                           pill_point_nbr: np.ndarray,
                           coor_to_pillar_id: np.ndarray,
                           min: np.ndarray,
                           step: np.ndarray,
                           max_ppp: int):
    """
        pcloud[nbpoints, 4]: array with points from point cloud (should be shuffled)
        pillars[nbpillars, 50, 4]: array with the final pcloud points per pillar (init as zero)
        pill_ind[nbpillars, 2]: x,y index for each pillar (init as zero)
        pill_point_nbr[nbpillars]: non zero points currently in the pillar
        coor_to_pillar_id[x_nbr, y_nbr,1]: with the xy bounds from the pillar get the pillar id (init with -1)

        returns: number of non zero pillars

    """
    N = pcloud.shape[0]
    pill_num = 0
    for i in range(N):
        point = pcloud[i]
        pil_coor = ((point[:2] - min) / step).astype(np.int32)
        pillar_id = coor_to_pillar_id[pil_coor[0], pil_coor[1]]
        # the pillar is not used yet
        if pillar_id == -1:
            pillar_id = pill_num
            coor_to_pillar_id[pil_coor[0], pil_coor[1]] = pillar_id
            pill_num += 1
            # set the correct bound so they can be transformed
            pill_ind[pillar_id] = pil_coor

        # non zero points in current pillar
        p_nbr = pill_point_nbr[pillar_id]
        if p_nbr < max_ppp:
            pillars[pillar_id, p_nbr] = point
            pill_point_nbr[pillar_id] += 1

    return pill_num

This method sets up all the necessary tensors and arrays for the numba method. This improves performance, since
setting up large tensors can be very time consuming in numba.

In [13]:
#export
def get_points_in_pillars(pcloud: torch.Tensor,
                          pillars_cfg: dict,
                          shuffle: bool = True,
                          dev: str = "cuda:0"):
    """
        not shuffling saves about 7ms
    """
    logger.info("Selecting points in pillar...")
    logger.debug(f"pcloud: {pcloud}{pcloud.shape}")

    if shuffle:
        pcloud = pcloud[torch.randperm(pcloud.shape[0])]

    min = np.array((float(pillars_cfg["x_min"]), float(pillars_cfg["y_min"])), dtype=np.float32)
    max = np.array((float(pillars_cfg["x_max"]), float(pillars_cfg["y_max"])), dtype=np.float32)
    step = np.array((float(pillars_cfg["x_step"]), float(pillars_cfg["y_step"])), dtype=np.float32)
    max_ppp = pillars_cfg.getfloat("max_points_per_pillar")
    max_pil = pillars_cfg.getint("max_pillars")
    n_pil = ((max - min) / step).astype(np.int32)
    n_pil_all = n_pil[0] * n_pil[1]

    pillars = np.zeros([n_pil_all, int(max_ppp), 4], dtype=np.float32)
    pill_ind = np.zeros([n_pil_all, 2], dtype=np.float32)
    pill_point_nbr = np.zeros([n_pil_all], dtype=np.int32)
    coor_to_pillar_id = -1 * np.ones(n_pil, dtype=np.int32)
    pcloud = remove_invalid_points(pcloud, pillars_cfg).to("cpu").numpy()

    logger.debug("Running jit method...")
    _get_points_in_pillars(pcloud, pillars, pill_ind, pill_point_nbr, coor_to_pillar_id, min, step, max_ppp)

    pill_tens = torch.from_numpy(pillars[:max_pil]).cuda()
    pill_ind_tens = torch.from_numpy(pill_ind[:max_pil]).cuda()

    logger.debug(f"Point selection complete.\n"
                 f"pill_tens: {pill_tens}{pill_tens.shape},\n"
                 f"pill_ind_tens: {pill_ind_tens}{pill_ind_tens.shape}")
    return pill_tens, pill_ind_tens

##### Testing

In [14]:
min = np.array([0,-50.0])
max = np.array([100, 50.0])
step = np.array([1, 1.0])
max_ppp = 50.0
pillars = np.zeros([251000, 100, 4])
pill_ind = np.zeros([251000, 2])
pill_point_nbr = np.zeros([251000], dtype=np.int16)
coor_to_pillar_id = -1*np.ones([501, 501], dtype=np.int16)
#time = time_method(_get_points_in_pillars, runs=10, kwargs={"pcloud": pc_vel, "pillars": None, "min": min, "step": step,
#                                                            "max_ppp": max_ppp, "pillars": pillars, "pill_ind": pill_ind, "pill_point_nbr": pill_point_nbr,
#                                                            "coor_to_pillar_id": coor_to_pillar_id})

#time, pillars, pillars.shape, pill_point_nbr, pill_point_nbr.shape, pill_ind, pill_ind.shape, coor_to_pillar_id, coor_to_pillar_id.shape

In [15]:
pillars_cfg = read_config()['pillars']
pill_tens, pill_ind_tens = get_points_in_pillars(pc, pillars_cfg)

pill_tens, pill_tens.shape, pill_ind_tens, pill_ind_tens.shape

(tensor([[[ 6.6714e+01,  2.2073e+00, -8.0537e-01,  1.0000e+00],
          [ 6.6683e+01,  2.1124e+00, -8.0623e-01,  1.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          ...,
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],
 
         [[ 3.6479e+00,  2.5852e+00, -8.5051e-01,  1.0000e+00],
          [ 3.6394e+00,  2.5614e+00, -6.5672e-01,  1.0000e+00],
          [ 3.5374e+00,  2.5642e+00, -8.4607e-01,  1.0000e+00],
          ...,
          [ 3.6744e+00,  2.6128e+00, -7.3366e-01,  1.0000e+00],
          [ 3.6559e+00,  2.6100e+00, -8.4745e-01,  1.0000e+00],
          [ 3.6733e+00,  2.5622e+00, -8.5713e-01,  1.0000e+00]],
 
         [[ 2.4180e+01, -1.2483e+01, -6.0869e-02,  1.0000e+00],
          [ 2.4212e+01, -1.2501e+01,  4.0522e-02,  1.0000e+00],
          [ 2.4164e+01, -1.2543e+01,  5.7233e-03,  1.0000e+00],
    

##### Timing

In [16]:
pillars_cfg = read_config()['pillars']
time_method(get_points_in_pillars, runs=10, kwargs={"pcloud": pc, "pillars_cfg": pillars_cfg})

tensor(33.3347)


#### 02.2 - Pillar calc class
> This method calculates the necessary extra parameters for each point. Those parameters include the distance
> to the pillar mean as well as the distance to the pillar center.


In [17]:
#export
class PillarCalc():

    def __init__(self, pillars_cfg: dict):
        logger.debug("Initializing PillarCalc module...")
        self.pillars_cfg = pillars_cfg

    def _pillar_centers_from_index(self, xy_index: torch.tensor):
        """
            converts the pillar bounds into centers.  Pillars center shape must be (pillar_nbr, 2] with the
            last dimension being [x_min, y_min] for each pillar
        """
        logger.info("Calculating pillar_centers_from_index.")
        logger.debug(f"xy_index: {xy_index}{xy_index.shape}")

        min = torch.cuda.FloatTensor([self.pillars_cfg.getfloat('x_min'), self.pillars_cfg.getfloat('y_min')])
        step = torch.cuda.FloatTensor([self.pillars_cfg.getfloat('x_step'), self.pillars_cfg.getfloat('y_step')])
        z_center = torch.cuda.FloatTensor([(self.pillars_cfg.getfloat('z_max') - self.pillars_cfg.getfloat('z_min')) / 2.0])

        # bring z center on shape from xy_min for concatenation
        z_center = z_center.unsqueeze(0).expand(xy_index.shape[0], -1)

        # The actual pillar boundaries (min has to be added again)
        xy_index = xy_index * step + min
        xy_index.add_(0.5 * step)
        xy_index = torch.cat((xy_index, z_center), dim=1)

        logger.debug(f"Center calculation complete.\n"
                     f"xy_center: {xy_index}{xy_index.shape},\n"
                     f"z_center: {z_center}{z_center.shape}")

        return xy_index

    def __call__(self, pillars: torch.Tensor, pillar_index: torch.Tensor, dt = torch.float32):
        """Returns the tensor with the given and all the calculated attributes.
            :param pillars:
            :param pillar_index:
            :param dt: datatype for torch tensors

            :returns:
        """
        logger.info("Calculating Pillars..")
        logger.debug(f"pillars: {pillars}{pillars.shape},\n"
                     f"pillar_index: {pillar_index}{pillar_index.shape}")

        # create mask for calculation because already zero padded
        centers = self._pillar_centers_from_index(pillar_index)
        mask = (pillars != 0)[:,:,:3]
        val = pillars[:,:,:3]

        # calculate the mean
        mean = val.mul_(mask).sum(dim=1)
        mean /= mask.sum(dim=1)

        # calculate difference to mean
        mean = mean.unsqueeze(1).expand(-1, val.shape[1], -1).clone()
        mean *= -1 * mask
        mean += val

        # replace the NaN with zeros
        mean[torch.isnan(mean)] = 0

        # calculate difference to centers
        centers = centers.unsqueeze(1).expand(-1, val.shape[1], -1).clone()
        centers *= -1 * mask
        centers += val
        centers = centers[:,:,:2]

        logger.debug(f"Pillar calculation complete.\n"
                     f"pillars: {pillars}{pillars.shape},\n"
                     f"diff_to_mean: {mean}{mean.shape},\n"
                     f"diff_to_center: {centers}{centers.shape}")

        return torch.cat((pillars, mean, centers), dim=2)


#### 02.3 - The actual calculation method

In [18]:
#export
def calculate_pillars(pcloud : torch.tensor):
    """
        The actual pillar calculation

        :param pcloud: The point cloud from which to extract and calculate pillars

        :returns: List[Tensor(nb_attributes, nbr_pillars w. zero_padding, nb_points_in_pillars),
                        Tensor(nb_pillars, 2)]
                  First tensor containing the points sorted into their corresponding pillar
                  Second Tensor containing the unique index for each pillar to later identify the position
    """
    logger.info("Calculating Pillars...")
    if not torch.is_tensor(pcloud):
        raise ValueError("Tensor expected but not given.")

    pillars_cfg = read_config()['pillars']
    pillars, pillar_index = get_points_in_pillars(pcloud, pillars_cfg, shuffle=True)

    # create model and move to gpu
    model = PillarCalc(pillars_cfg)


    logger.info("Pillar calculation complete!")
    # permutation to receive correct pillar layout with (D,P,N)
    return model(pillars, pillar_index).permute(2,0,1), pillar_index.type(torch.LongTensor)

##### Testing

In [19]:
out = calculate_pillars(pc_vel)
out, out[0].shape, out[1].shape

tensor([0.1600, 0.1600], device='cuda:0') torch.Size([2])
tensor([  0.0000, -40.3200], device='cuda:0') torch.Size([2])
tensor([[ 24., 199.],
        [  9., 396.],
        [ 18., 235.],
        ...,
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.]], device='cuda:0') torch.Size([12000, 2])
tensor([[  3.8400,  -8.4800],
        [  1.4400,  23.0400],
        [  2.8800,  -2.7200],
        ...,
        [  0.0000, -40.3200],
        [  0.0000, -40.3200],
        [  0.0000, -40.3200]], device='cuda:0') torch.Size([12000, 2])
tensor([[  3.9200,  -8.4000],
        [  1.5200,  23.1200],
        [  2.9600,  -2.6400],
        ...,
        [  0.0800, -40.2400],
        [  0.0800, -40.2400],
        [  0.0800, -40.2400]], device='cuda:0') torch.Size([12000, 2])


((tensor([[[ 3.9110e+00,  3.9150e+00,  3.8750e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [ 1.5020e+00,  1.4990e+00,  1.5120e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [ 2.9330e+00,  3.0350e+00,  3.0100e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           ...,
           [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00]],
  
          [[-8.4350e+00, -8.4280e+00, -8.4650e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [ 2.3082e+01,  2.3061e+01,  2.3105e+01,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [-2.5860e+00, -2.6750e+00, -2.6900e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],

#### Timing

In [20]:
time_method(calculate_pillars, runs=10, kwargs={"pcloud": pc_vel})

tensor([0.1600, 0.1600], device='cuda:0') torch.Size([2])
tensor([  0.0000, -40.3200], device='cuda:0') torch.Size([2])
tensor([[ 17., 196.],
        [ 31., 286.],
        [  0., 191.],
        ...,
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.]], device='cuda:0') torch.Size([12000, 2])
tensor([[  2.7200,  -8.9600],
        [  4.9600,   5.4400],
        [  0.0000,  -9.7600],
        ...,
        [  0.0000, -40.3200],
        [  0.0000, -40.3200],
        [  0.0000, -40.3200]], device='cuda:0') torch.Size([12000, 2])
tensor([[  2.8000,  -8.8800],
        [  5.0400,   5.5200],
        [  0.0800,  -9.6800],
        ...,
        [  0.0800, -40.2400],
        [  0.0800, -40.2400],
        [  0.0800, -40.2400]], device='cuda:0') torch.Size([12000, 2])
tensor([0.1600, 0.1600], device='cuda:0') torch.Size([2])
tensor([  0.0000, -40.3200], device='cuda:0') torch.Size([2])
tensor([[ 66., 151.],
        [ 31., 285.],
        [ 40., 319.],
        ...,
        [  0.,   0.],
     

tensor(40.3198)

### Export notebooks

In [21]:

#hide
#from nbdev.export import notebook2script
#notebook2script()
